In [1]:
%pip -q install google-genai

In [2]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
from google import genai

client = genai.Client()

MODEL_VERSION = "gemini-2.0-flash"

In [4]:

from IPython.display import HTML, Markdown

result = client.models.generate_content(
    model= MODEL_VERSION,
    contents ='Quando é a proxima Imersao IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {result.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. A melhor maneira de saber sobre as próximas edições é:

*   **Acompanhar a página oficial da Alura:** Verifique regularmente a seção de imersões ou cursos de inteligência artificial no site da Alura.
*   **Assinar a newsletter da Alura:** Assim, você receberá informações sobre os próximos lançamentos e eventos.
*   **Seguir as redes sociais da Alura:** Eles costumam divulgar as novidades por lá.
*   **Participar da comunidade da Alura:** Fique de olho nos fóruns e grupos de discussão, onde os alunos compartilham informações sobre os próximos cursos e imersões.

In [6]:
result = client.models.generate_content(
    model=MODEL_VERSION,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools":[{"google_search": {}}]}
)

display(Markdown(f"Resposta:\n {result.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura já aconteceu entre os dias 12 e 16 de maio de 2025. As inscrições estiveram abertas até o dia 11 de maio de 2025.


In [7]:
# Instalar Framework ADK de agentes do Google
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [8]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import requests
import warnings

warnings.filterwarnings("ignore")

In [9]:

def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""

    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [10]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [12]:
# --- Agente 1: Buscador de Notícias Global ---

def agente_buscador(topico, data_de_hoje):
  buscador = Agent(
      name="agente_buscador",
      model= MODEL_VERSION,
      instruction= """
      Você é um assistente de pesquisa global. A sua tarefa é usar a ferramenta de busca do google (google_search)
      para recuperar as últimas notícias de lançamentos muitos relavantes sonre o tópico abaixo.
      Foque em no máximo em 5 lançamentos relevantes, com base na quantidade e entusiasmo das noticias sobre eles.
      Se um tema tiver poucas noticias ou reações entusiamadas, é possivel que ele não seja tão relevante assim e
      pode ser substituido por outro tema que tenha mais relevancia.
      Esses lancamentos devem ser atuais, de no máximo um mês antes da data de hoje, e caso estejam em ingles ou
      qualquer idioma, traduza  para o português brasileiro antes de retornar a notícia.
      """,
      description="Agente que busca notícias sobre um determinado tópico no Google",
      tools =[google_search],
  )


  entrada_do_agente_buscador = f"Tópico {topico}\nData de Hoje: {data_de_hoje}"
  resposta_do_agente_buscador = call_agent(buscador, entrada_do_agente_buscador)
  return resposta_do_agente_buscador


In [20]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um planejador de conteúdo, especialista em redes socias. Com base na lista lançamentos mais recentes e relevantes
        buscador, você deve:
        usar a ferramenta do Google(google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um post sobre
        cada um deles. Você também pode usar o (google_search) para encontrarmais informações sobre os temas e aprovofundar.
        Ao final, você pode escolher o tema mais relevante entre eles com base nas suas pesquisas e retornar esse tema, seus pontos
        mais relevantes, e a plane com os assuntos a serem abordados no post que será escrito posterioermente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [21]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [22]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [30]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")


topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")


if not topico:
  print("Digite por favor  o tópico desejado.")
else:
  print(f"ótimo, Vamos criar o post sobre novidades em {topico}")

  lacamentos = agente_buscador(topico, data_de_hoje)
  plano_de_post = agente_planejador(topico, lacamentos)
  rascunho_de_post = agente_planejador(topico, plano_de_post)
  post_final = agente_revisor(topico,rascunho_de_post)

display(to_markdown(post_final))

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: modelos de pipas
ótimo, Vamos criar o post sobre novidades em modelos de pipas


Exception in thread Thread-66 (_asyncio_thread_main):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 138, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
  File "/usr/lib/python3.11/asyncio/runners.py", line 190, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 654, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 126, in _invoke_run_async
    async for event in self.run_async(
  File "/u